In [16]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 3.8MB 44.6MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [5]:
import numpy as np
import re
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import os
import unicodedata
import urllib3
import zipfile

In [14]:
text = pd.read_excel('/content/kor.xlsx', encoding='CP949')
text

,mid_sid,ko,en
0,1,나는 매일 저녁 배트를 만나러 다락방으로 가요.,I go to the attic every evening to meet Bat.
1,2,선생님 이문장이 이해가 안 가요.,"Sir, I don't understand this sentence here."
2,3,컴퓨터를 시작하면 시간이 너무 빠르게 가요.,Time flies when you start using the computer.
3,4,나는 오늘 자정에 한국으로 돌아 가요.,I'm going back to Korea today at midnight.
4,5,나는 일어나자마자 화장실에 가요.,I go to bathroom as soon as I wake up.
...,...,...,...
74995,74996,나의 고민은 학교가 멀어서 통학하기 힘들어.,My worry is commuting to school because it's t...
74996,74997,난 지금 내고양이때문에 충분히 힘들어.,I am going under enough difficulties because o...
74997,74998,나와 대화가 어려운 것이 많이 힘들어?,Is having difficulties in talking with me too ...
74998,74999,하루에 한번 연락하는게 그렇게 힘들어?,Is it that difficult to call once a day?


In [15]:
df = text[['ko','en']]

df.shape

(75000, 2)

In [16]:
df.head()

,ko,en
0,나는 매일 저녁 배트를 만나러 다락방으로 가요.,I go to the attic every evening to meet Bat.
1,선생님 이문장이 이해가 안 가요.,"Sir, I don't understand this sentence here."
2,컴퓨터를 시작하면 시간이 너무 빠르게 가요.,Time flies when you start using the computer.
3,나는 오늘 자정에 한국으로 돌아 가요.,I'm going back to Korea today at midnight.
4,나는 일어나자마자 화장실에 가요.,I go to bathroom as soon as I wake up.


## 전처리

유니코드(Unicode) 문자열은 문자를 Unicode로 취급하기 때문에 영어나 한국어등의 ASCII 문자로 되어있어도 한 개 문자를 하나씩 다룹니다.

즉, 일반 문자열은 바이트 단위로 처리 하지만, 유니코드 문자열에서는 문자 단위로 처리하게 됩니다.

한글을 글자 깨짐 없이 사용하고 싶은 경우에는 유니코드 문자열을 사용하면 됩니다.

In [17]:
def unicode_to_ascii(s): 
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

In [18]:
def preprocess_sentence(sent):
    # 위에서 구현한 함수를 내부적으로 호출
    # sent = unicode_to_ascii()

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣!.?]+", r" ", sent)

    sent = re.sub(r"\s+", " ", sent)
    return sent

In [19]:
# 전처리 테스트
# 문자열 앞에 [u]또는 [U]를 붙여주면 유니코드 문자열이 됩니다.

kor_sent1 = u"저녁 먹었어?"
eng_sent1 = u"Have you had dinner?"

print(preprocess_sentence(eng_sent1))
print(preprocess_sentence(kor_sent1))

Have you had dinner ?
저녁 먹었어 ?


- 전체 데이터에서 33,000개의 샘플만 불러오되, 모든 전처리를 수행하는 함수를 만듭니다.   
- 또한 훈련 과정에서 교사 강요(Teacher Forcing)을 사용할 예정이므로, 훈련 시 사용할 디코더의 입력 시퀀스와 실제값에 해당되는 출력 시퀀스를 따로 분리하여 저장합니다.   
- 입력 시퀀스에는 시작을 의미하는 토큰인 \<sos\>를 추가하고, 출력 시퀀스에는 종료를 의미하는 토큰인 \<eos\>를 추가합니다  

In [20]:
len(df)

75000

In [78]:
def load_preprocessed_data(df):
    encoder_input, decoder_input, decoder_target = [], [], []
    for i in range(len(df)):
      src_line = df.iloc[i][0].strip()
      tar_line = df.iloc[i][1].strip()

      # source 데이터 전처리
      src_line_input = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_input = [w for w in ("<sos> " + tar_line).split()]
      tar_line_target = [w for w in (tar_line + " <eos>").split()]

      encoder_input.append(src_line_input[::-1])
      decoder_input.append(tar_line_input)
      decoder_target.append(tar_line_target)

      # if i == num_samples - 1:
      #     break

    return encoder_input, decoder_input, decoder_target

In [79]:
# encoder_input, decoder_input, decoder_target
sents_kor_in, sents_eng_in, sents_eng_out = load_preprocessed_data(df)

In [80]:
print(sents_kor_in[-5:])
print(sents_eng_in[-5:])
print(sents_eng_out[-5:])

[['.', '힘들어', '통학하기', '멀어서', '학교가', '고민은', '나의'], ['.', '힘들어', '충분히', '내고양이때문에', '지금', '난'], ['?', '힘들어', '많이', '것이', '어려운', '대화가', '나와'], ['?', '힘들어', '그렇게', '연락하는게', '한번', '하루에'], ['.', '힘들죠', '많이', '즐기기엔', '스포츠를', '아이들이', '어린']]
[['<sos>', 'My', 'worry', 'is', 'commuting', 'to', 'school', 'because', 'it', 's', 'too', 'far', '.'], ['<sos>', 'I', 'am', 'going', 'under', 'enough', 'difficulties', 'because', 'of', 'my', 'cat', '.'], ['<sos>', 'Is', 'having', 'difficulties', 'in', 'talking', 'with', 'me', 'too', 'hard', 'for', 'you', '?'], ['<sos>', 'Is', 'it', 'that', 'difficult', 'to', 'call', 'once', 'a', 'day', '?'], ['<sos>', 'It', 'is', 'difficult', 'for', 'young', 'children', 'to', 'enjoy', 'sports', '.']]
[['My', 'worry', 'is', 'commuting', 'to', 'school', 'because', 'it', 's', 'too', 'far', '.', '<eos>'], ['I', 'am', 'going', 'under', 'enough', 'difficulties', 'because', 'of', 'my', 'cat', '.', '<eos>'], ['Is', 'having', 'difficulties', 'in', 'talking', 'with', 'me', 'too', 'har

In [91]:
# 인코딩
tokenizer_kor = Tokenizer()
tokenizer_kor.fit_on_texts(sents_kor_in)
encoder_input = tokenizer_kor.texts_to_sequences(sents_kor_in)

tokenizer_eng = Tokenizer(filters="", lower=True)
tokenizer_eng.fit_on_texts(sents_eng_in)
tokenizer_eng.fit_on_texts(sents_eng_out)
decoder_input = tokenizer_eng.texts_to_sequences(sents_eng_in)
decoder_target = tokenizer_eng.texts_to_sequences(sents_eng_out)

In [92]:
# 패딩
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [93]:
encoder_input.shape, decoder_input.shape, decoder_target.shape

((75000, 15), (75000, 19), (75000, 19))

In [94]:
src_vocab_size = len(tokenizer_kor.word_index) + 1
tar_vocab_size = len(tokenizer_eng.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}, 영어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

한국어 단어 집합의 크기 : 85237, 영어 단어 집합의 크기 : 19285


In [95]:
encoder_input

array([[    1,   211, 29411, ...,     0,     0,     0],
       [    1,   211,    72, ...,     0,     0,     0],
       [    1,   211,   977, ...,     0,     0,     0],
       ...,
       [    2,  1307,    25, ...,     0,     0,     0],
       [    2,  1307,   232, ...,     0,     0,     0],
       [    1, 85236,    25, ...,     0,     0,     0]], dtype=int32)

In [96]:
src_to_index = tokenizer_kor.word_index
index_to_src = tokenizer_kor.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_eng.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_eng.index_word # 훈련 후 결과 비교할 때 사용

In [97]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[56209 53138  4299 ... 20058 38635 10770]


In [98]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [100]:
encoder_input[30997], decoder_input[30997], decoder_target[30997]

(array([    1, 43566,  7599,  9645, 43567,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0], dtype=int32),
 array([    2,     8,  3198,    13,  5983,     9,     8,  1603,    17,
         7547, 12772,    13,    61,     1,     0,     0,     0,     0,
            0], dtype=int32),
 array([    8,  3198,    13,  5983,     9,     8,  1603,    17,  7547,
        12772,    13,    61,     1,     3,     0,     0,     0,     0,
            0], dtype=int32))

In [101]:
n_of_val = int(len(df)*0.1)
print(n_of_val)

7500


In [102]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [103]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(67500, 15)
(67500, 19)
(67500, 19)
(7500, 15)
(7500, 19)
(7500, 19)


In [104]:
encoder_input_train

array([[    1,    24,   558, ...,     0,     0,     0],
       [    1,  4353,  1552, ...,     0,     0,     0],
       [    1,    33,    95, ...,     0,     0,     0],
       ...,
       [    2,  2555, 71841, ...,     0,     0,     0],
       [    1,  3329,   569, ...,     0,     0,     0],
       [    1,  2439, 51934, ...,     0,     0,     0]], dtype=int32)

## 기계 번역기 만들기

In [105]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [107]:
src_vocab_size

85237

In [108]:
latent_dim = 100

In [109]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True, return_sequence=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

In [110]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [111]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [138]:
# learning rate 조절하기 
def scheduler(epoch, lr):
  if epoch < 5:
    return lr
  else:
    return lr/2*(epoch-5)

In [135]:
from tensorflow.keras.optimizers import SGD
epochs = 50
learning_rate = 0.7
model.compile(SGD(learning_rate=0.75), loss='sparse_categorical_crossentropy', metrics = ['acc'])

In [136]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
callbacks = [
             EarlyStopping(monitor ='val_acc', patience = 3),
             LearningRateScheduler(scheduler)
             ]

In [137]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 30, callbacks = callbacks)

Epoch 1/30
528/528 [==============================] - 58s 110ms/step - loss: 1.9493 - acc: 0.6717 - val_loss: 2.3719 - val_acc: 0.6286
Epoch 2/30
528/528 [==============================] - 56s 107ms/step - loss: 1.7748 - acc: 0.6971 - val_loss: 2.4344 - val_acc: 0.6201
Epoch 3/30
528/528 [==============================] - 56s 107ms/step - loss: 1.7224 - acc: 0.7052 - val_loss: 2.4109 - val_acc: 0.6252
Epoch 4/30
528/528 [==============================] - 56s 107ms/step - loss: 1.6909 - acc: 0.7105 - val_loss: 2.4521 - val_acc: 0.6220


## seq2seq 기계 번역기 동작시키기

keras model의 뜻

In [139]:
from keras.models import Model
from keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

model.summary()
# input 'a'와, output 'b'를 구성하기 위한 모든 layer를 자동으로 구성함

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
Total params: 1,056
Trainable params: 1,056
Non-trainable params: 0
_________________________________________________________________


In [140]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, None)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, None, 100)         8523700   
_________________________________________________________________
masking_4 (Masking)          (None, None, 100)         0         
_________________________________________________________________
lstm_4 (LSTM)                [(None, 100), (None, 100) 80400     
Total params: 8,604,100
Trainable params: 8,604,100
Non-trainable params: 0
_________________________________________________________________


In [141]:
# # 디코더
# decoder_inputs = Input(shape=(None,))
# dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
# dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
# dec_masking = Masking(mask_value=0.0)(dec_emb)

# # 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
# decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# # 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
# decoder_outputs, _, _ = decoder_lstm(dec_masking,
#                                      initial_state=encoder_states)

# # 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
# decoder_dense = Dense(tar_vocab_size, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

In [142]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [143]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [144]:
decoder_model.summary()

Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    1928500     input_11[0][0]                   
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 100)]        0                                            
______________________________________________________________________________________

In [145]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [146]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if (i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if ((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

In [148]:
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index][::-1]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[:-5])
  print("\n")

원문 :  그리고 국물에는 얼음이 들어가있어서 시원했어요 . 
번역문 : and the soup was so cooling because it had ice in it . 
예측문 :  and it was not easy but it s a little bit of 


원문 :  결국 이 프로젝트는 아이들에게 트라우마가 돼요 . 
번역문 : eventually this project becomes trauma for the children . 
예측문 :  this lot of addition should make our own leas


원문 :  타미가 저를 좋아하는지는 중요하지 않아요 . 
번역문 : whether tammy likes me is not the point . 
예측문 :  not true is not the same as a gift . 


원문 :  오히려 내 영어 실력이 부끄러워지는 순간이에요 . 
번역문 : it was the moment my english skill rather felt embarrassing . 
예측문 :  it was a good memory that i m afraid of good mem


원문 :  한국의 은행으로부터 돈을 찾는데 필요해요 . 
번역문 : it is needed to withdraw money from banks in korea . 
예측문 :  it is necessary to use your own country s own . 


